# Sesión 1 – Apartado 3: Pipelines en Scikit-learn

**Duración recomendada:** ~50 min

**Objetivos:**
- Comprender el uso de `Pipeline` y `ColumnTransformer`.
- Encadenar preprocesamiento y modelos.
- Facilitar reproducibilidad y despliegue.


## 1) Cargar el dataset

In [ ]:
import pandas as pd
df = pd.read_csv('data/mini_titanic.csv')
df.head()

## 2) Definir variables numéricas y categóricas

In [ ]:
num_cols = ['age', 'fare']
cat_cols = ['sex', 'class']
target = 'survived'

X = df[num_cols + cat_cols]
y = df[target]

## 3) ColumnTransformer

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import SimpleImputer, StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

# Pipeline numérico
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Pipeline categórico
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# ColumnTransformer
preprocessor = ColumnTransformer([
    ('num', num_pipe, num_cols),
    ('cat', cat_pipe, cat_cols)
])

preprocessor

## 4) Pipeline completo con modelo

In [ ]:
clf = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LogisticRegression(max_iter=500))
])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

clf.fit(X_train, y_train)
print('Accuracy en test:', clf.score(X_test, y_test))

## 5) Inspeccionar el pipeline

In [ ]:
print(clf.named_steps)
print('\nTransformaciones aplicadas a un batch de datos:')
import numpy as np
Xt = clf.named_steps['preprocessor'].transform(X_train)
print('Shape transformado:', Xt.shape)


## 6) Guardar y cargar un pipeline entrenado

In [ ]:
import joblib

# Guardar
joblib.dump(clf, 'data/titanic_pipeline.joblib')

# Cargar
clf_loaded = joblib.load('data/titanic_pipeline.joblib')
print('Accuracy cargado:', clf_loaded.score(X_test, y_test))

## 7) Ejercicios

1. Sustituye el modelo `LogisticRegression` por un `RandomForestClassifier` dentro del pipeline.
2. Cambia la estrategia de imputación de los numéricos de `mean` a `median` y evalúa el impacto.
3. Añade un paso opcional de reducción dimensional con `PCA` después del preprocesamiento.
4. Discute: ¿qué ventajas tiene usar `Pipeline` frente a aplicar cada paso manualmente?
